In [1]:
pip install pysal

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mgwr

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import pysal as ps
from mgwr.sel_bw import Sel_BW
from mgwr.gwr import GWR, MGWR

/Users/tiffalth/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<string>:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).


In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_masked.csv')

clean_train = train.drop(['year', 'cname', 'path', 'DHSID_EA' , 'n_water'], axis = "columns")
clean_train['urban'] = clean_train['urban'].replace({'R': 1, 'U': 0})

clean_test = test.drop(['year', 'cname', 'path', 'DHSID_EA'], axis = "columns")
clean_test['urban'] = test['urban'].replace({'R': 1, 'U': 0})

In [12]:
clean_train

,lat,lon,n_asset,asset_index,water_index,cluster_id,adm1dhs,urban
0,9.165413,92.742696,22.0,2.650768,5.000000,10004,1,1
1,8.307356,93.093792,22.0,2.157784,5.000000,10005,1,1
2,7.016968,93.893226,21.0,1.832751,4.619048,10007,1,1
3,9.194938,92.800432,22.0,2.746096,5.000000,10016,1,1
4,8.055606,93.543892,22.0,2.581869,5.000000,10018,1,1
...,...,...,...,...,...,...,...,...
18717,17.184983,79.989186,22.0,0.629311,4.272727,360473,36,1
18718,17.465781,78.418820,20.0,2.704372,5.000000,360475,36,0
18719,17.939231,79.512509,17.0,1.315361,4.882353,360480,36,1
18720,17.420175,78.491013,23.0,2.920893,5.000000,360481,36,0


In [14]:
# Preparing the independent variables
g_x = clean_train[['asset_index', 'cluster_id', 'adm1dhs', 'urban']].values
g_x[0:5]

array([[2.65076817e+00, 1.00040000e+04, 1.00000000e+00, 1.00000000e+00],
       [2.15778423e+00, 1.00050000e+04, 1.00000000e+00, 1.00000000e+00],
       [1.83275082e+00, 1.00070000e+04, 1.00000000e+00, 1.00000000e+00],
       [2.74609580e+00, 1.00160000e+04, 1.00000000e+00, 1.00000000e+00],
       [2.58186909e+00, 1.00180000e+04, 1.00000000e+00, 1.00000000e+00]])

In [19]:
g_y = clean_train['water_index'].values.reshape((-1,1))
g_y[0:5]

array([[5.        ],
       [5.        ],
       [4.61904762],
       [5.        ],
       [5.        ]])

In [9]:
u = train['lat']
v = train['lon']
g_coords = list(zip(u,v))
g_coords[0:5]

[(9.16541348453, 92.7426958972),
 (8.30735625773, 93.0937924407),
 (7.01696797689, 93.8932263934),
 (9.1949376382, 92.8004316003),
 (8.05560574626, 93.5438918498)]

In [24]:
# Calibrating the GWR model
gwr_selector = Sel_BW(g_coords, g_y, g_x, kernel='gaussian')
gwr_bw = gwr_selector.search()
print('GWR bandwidth =', gwr_bw)

/Users/tiffalth/opt/anaconda3/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=5.48888e-17): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/tiffalth/opt/anaconda3/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=6.54908e-17): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/tiffalth/opt/anaconda3/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=9.08161e-17): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/tiffalth/opt/anaconda3/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=3.82661e-17): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/tiffalth/opt/anaconda3/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=4.46688e-17): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)


GWR bandwidth = 51.0


/Users/tiffalth/opt/anaconda3/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=8.65774e-18): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/tiffalth/opt/anaconda3/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.12119e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/tiffalth/opt/anaconda3/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=7.79116e-21): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)


In [78]:
g_coords = np.asarray(g_coords)
g_coords.shape

(18722, 2)

In [60]:
model = GWR(g_coords, g_y, g_x, gwr_bw, kernel='gaussian')

In [61]:
u = test['lat']
v = test['lon']
g_coords_test = list(zip(u,v))
g_coords_test[0:5]

[(9.22090276937, 92.7815301604),
 (7.02841029446, 93.8834303952),
 (12.3714479459, 92.7836649333),
 (11.7273040922, 92.7192571031),
 (9.18531014236, 92.777644986)]

In [62]:
test_x = clean_test[['asset_index', 'cluster_id', 'adm1dhs', 'urban']].values
test_x[0:5]

array([[2.72181171e+00, 1.00090000e+04, 1.00000000e+00, 1.00000000e+00],
       [2.28727907e+00, 1.00110000e+04, 1.00000000e+00, 1.00000000e+00],
       [6.77109270e-01, 1.00170000e+04, 1.00000000e+00, 1.00000000e+00],
       [1.79368300e+00, 1.00440000e+04, 1.00000000e+00, 1.00000000e+00],
       [2.75816834e+00, 1.00600000e+04, 1.00000000e+00, 1.00000000e+00]])

In [75]:
test_x1 = np.array(clean_test)

test_coords = test.drop(['year', 'cname', 'path', 'DHSID_EA', 'n_asset', 'asset_index', 'cluster_id', 'adm1dhs', 'urban'], axis='columns')
g_coords_pls = np.array(test_coords)

In [69]:
g_coords_test1 = np.asarray(g_coords_test)
g_coords_test1[0:5]

array([[ 9.22090277, 92.78153016],
       [ 7.02841029, 93.8834304 ],
       [12.37144795, 92.78366493],
       [11.72730409, 92.7192571 ],
       [ 9.18531014, 92.77764499]])

In [76]:
results = model.predict(g_coords_pls, test_x1)

AttributeError: 'list' object has no attribute 'shape'